# 2.5 Hypothesis Testing 

In this notebook I will redo all of my hypothesis testing work to expand the scope and usefulness of identified features. I will remove those features with pvalue < 0.05 and build a linear regression to predict % close. I will also build a logistic regression to predict the sign(%close). 

#### Steps

1) Load a dataset  
2) Perform transformations  
3) Test features and remove, retest  
4) Build regressions  

In [1]:
from sine_modules import *
import pandas as pd
import numpy as np
import statsmodels.api as sm
import matplotlib.pyplot as plt

pd.set_option('display.max_columns', 200)
pd.set_option('display.max_rows', 100)

In [2]:
pd.__version__

'1.1.1'

In [3]:
stock = 'RRR'
data_dir = './data/prepared/august25screenfixed/'
suffix = '.pickle'

df = load_set(stock, data_dir, suffix)

In [4]:
df.head()

,open,high,low,close,volume,datetime,date,hour,minute,min_num,SYMBOL,prev_close,diff_1,pct_change,log_return,%open,mesa_open,open_amp,open_omega,open_phase,open_offset,open_freq,open_period,open_maxcov,open_sin,time,open_angle,open_rad,open_rad2,open_sin2,open_cos2,open_cos,open_tan,open_tan2,open_xsinx,open_xcosx,open_sinxcosx,open_xsinxcosx,open_xsinx2,open_xcosx2,open_sinxcosx2,open_xsinxcosx2,open_xtanx,open_xtanx2,%high,mesa_high,high_amp,high_omega,high_phase,high_offset,high_freq,high_period,high_maxcov,high_sin,high_angle,high_rad,high_rad2,high_sin2,high_cos2,high_cos,high_tan,high_tan2,high_xsinx,high_xcosx,high_sinxcosx,high_xsinxcosx,high_xsinx2,high_xcosx2,high_sinxcosx2,high_xsinxcosx2,high_xtanx,high_xtanx2,%low,mesa_low,low_amp,low_omega,low_phase,low_offset,low_freq,low_period,low_maxcov,low_sin,low_angle,low_rad,low_rad2,low_sin2,low_cos2,low_cos,low_tan,low_tan2,low_xsinx,low_xcosx,low_sinxcosx,low_xsinxcosx,low_xsinx2,low_xcosx2,low_sinxcosx2,low_xsinxcosx2,low_xtanx,low_xtanx2,%close,mesa_close,close_amp,close_omega,close_phase,close_offset,close_freq,close_period,close_maxcov,close_sin,close_angle,close_rad,close_rad2,close_sin2,close_cos2,close_cos,close_tan,close_tan2,close_xsinx,close_xcosx,close_sinxcosx,close_xsinxcosx,close_xsinx2,close_xcosx2,close_sinxcosx2,close_xsinxcosx2,close_xtanx,close_xtanx2,decision,D2
8845,17.28,17.28,17.28,17.28,100,2020-08-25 22:48:00,2020-08-25,22,48,1368,RRR,17.30,-0.02,-0.001156,-0.001157,-0.001156,0.490588,0.001894,2.999950,141.151213,-0.000400,0.477457,2.094430,2.221737e+05,-0.001211,8845,26675.705720,3.584091,4.061547,-0.001907,-0.605856,-0.903685,0.473836,1.313140,0.000001,0.001045,0.001095,-0.000001,2.204898e-06,0.000700,0.001156,-1.335851e-06,-0.000548,-0.001518,-0.001156,0.490591,-0.001940,2.802241,-332.993391,-0.000401,0.445991,2.242200,2.440428e+05,0.001482,24452.828207,4.954177,5.400168,0.001098,0.634822,0.239439,-4.054941,-1.217125,-0.000002,-0.000277,0.000355,-4.103498e-07,-0.000001,-0.000734,0.000697,-8.057384e-07,0.004688,0.001407,-0.001156,0.490588,0.002575,2.990564,224.104713,-0.000389,0.475963,2.101003,1.440349e+05,-0.001352,26675.646515,3.524886,4.000850,-0.002340,-0.653000,-0.927438,0.403237,1.159812,0.000002,0.001072,0.001254,-0.000001,0.000003,0.000755,0.001528,-0.000002,-0.000466,-0.001341,-0.001156,0.490591,-0.002573,2.833824,-611.936535,-0.000390,0.451017,2.217211,1.726659e+05,0.001653,24453.239631,5.365601,5.816618,0.000767,0.893118,0.607740,-1.306702,-0.503655,-0.000002,-0.000703,0.001004,-0.000001,-8.865898e-07,-0.001033,0.000685,-7.918291e-07,0.001511,0.000582,0.0,0.0
8844,17.30,17.30,17.30,17.30,200,2020-08-25 22:44:00,2020-08-25,22,44,1364,RRR,17.44,-0.14,-0.008028,-0.008060,-0.002882,0.489934,0.001881,1.287313,-270.196033,-0.000346,0.204882,4.880852,2.322576e+05,-0.000633,8844,11114.801657,6.130034,6.334916,-0.000249,0.998662,0.988295,-0.154360,0.051777,0.000002,-0.002848,-0.000626,0.000002,7.176933e-07,-0.002878,-0.000249,7.167332e-07,0.000445,-0.000149,-0.008028,0.488493,-0.002108,2.808354,-387.061715,-0.000335,0.446963,2.237320,1.819360e+05,-0.002103,24450.020219,2.146189,2.593152,-0.001434,-0.853339,-0.544164,-1.541776,-0.610962,0.000017,0.004368,0.001145,-9.187537e-06,0.000012,0.006850,0.001223,-9.820883e-06,0.012377,0.004905,-0.002882,0.489934,0.002457,3.001820,124.667322,-0.000327,0.477755,2.093125,2.107880e+05,0.001151,26672.767225,0.645596,1.123351,0.001888,0.432664,0.798741,0.753279,2.083731,-0.000003,-0.002302,0.000919,-0.000003,-0.000005,-0.001247,0.000817,-0.000002,-0.002171,-0.006005,-0.008028,0.488493,-0.002719,2.838034,-649.214703,-0.000325,0.451687,2.213922,1.408935e+05,-0.001990,24450.356426,2.482395,2.934083,-0.000885,-0.978547,-0.790484,-0.774819,-0.210541,0.000016,0.006346,0.001573,-0.000013,7.103888e-06,0.007855,0.000866,-6.951488e-06,0.006220,0.001690,0.0,0.0
8843,17.35,17.44,17.35,17.44,25,2020-08-25 22:36:00,2020-08-25,22,36,1356,RRR,17.35,0.09,0.005187,0.005174,0.000000,0.496402,0.004198,3.106833,-802.280898,-0

In [5]:
df['datetime'].shift(-1)

8845   2020-08-25 22:44:00
8844   2020-08-25 22:36:00
8843   2020-08-25 22:34:00
8842   2020-08-25 22:28:00
8841   2020-08-25 22:12:00
               ...        
4      2020-07-27 13:32:00
3      2020-07-27 13:31:00
2      2020-07-27 13:30:00
1      2020-07-27 12:42:00
0                      NaT
Name: datetime, Length: 8846, dtype: datetime64[ns]

In [7]:
df['open'].rolling(30).mean()

8845          NaN
8844          NaN
8843          NaN
8842          NaN
8841          NaN
          ...    
4       10.602963
3       10.609980
2       10.616647
1       10.624980
0       10.639313
Name: open, Length: 8846, dtype: float64

In [8]:
indexer = pd.api.indexers.FixedForwardWindowIndexer(window_size=2)
df['open'].rolling(indexer).mean()

8845    17.290
8844    17.325
8843    17.350
8842    17.335
8841    17.320
         ...  
4       10.655
3       10.700
2       10.785
1       10.925
0          NaN
Name: open, Length: 8846, dtype: float64

In [5]:
cols = ['open','close','high','low']

# sma, ema, high, low
# diff from 
# (actual price - high/low/sma/ema) / high/low/sma/ema

stat_cols = ['_sma30',
            '_sma50',
            '_std30',
            '_std50',
            '_var30',
            '_var50',
            '_min30',
            '_min50',
            '_max30',
            '_max50',
            '_skew30',
            '_skew50',
            '_kurt30',
            '_kurt50',]
#             '_ewmmean30',
#             '_ewmmean50']


for col in cols:
    for i in range(1,61):
        df[f'%{col}_{i}'] = df[f'%{col}'].shift(-i)
    
    indexer = pd.api.indexers.FixedForwardWindowIndexer(window_size=30)
    
    df[f'{col}_sma30'] = df[col].rolling(indexer).mean()
    df[f'{col}_std30'] = df[col].rolling(indexer).std()
    df[f'{col}_var30'] = df[col].rolling(indexer).var()
    df[f'{col}_min30'] = df[col].rolling(indexer).min()
    df[f'{col}_max30'] = df[col].rolling(indexer).max()
    df[f'{col}_skew30'] = df[col].rolling(indexer).skew()
    df[f'{col}_kurt30'] = df[col].rolling(indexer).kurt()
    #df[f'{col}_ewmmean30'] = df[col].ewm(indexer).mean()
    
    indexer = pd.api.indexers.FixedForwardWindowIndexer(window_size=50)
    
    df[f'{col}_sma50'] = df[col].rolling(indexer).mean()
    df[f'{col}_std50'] = df[col].rolling(indexer).std()
    df[f'{col}_var50'] = df[col].rolling(indexer).var()
    df[f'{col}_min50'] = df[col].rolling(indexer).min()
    df[f'{col}_max50'] = df[col].rolling(indexer).max()
    df[f'{col}_skew50'] = df[col].rolling(indexer).skew()
    df[f'{col}_kurt50'] = df[col].rolling(indexer).kurt()
    #df[f'{col}_ewmmean50'] = df[col].ewm(indexer).mean()

    for stat_col in stat_cols:
        df[f'%{col}{stat_col}'] = (df[f'{col}'] - df[f'{col}{stat_col}']) / df[f'{col}{stat_col}']
        
        
df.head()

,open,high,low,close,volume,datetime,date,hour,minute,min_num,SYMBOL,prev_close,diff_1,pct_change,log_return,%open,mesa_open,open_amp,open_omega,open_phase,open_offset,open_freq,open_period,open_maxcov,open_sin,time,open_angle,open_rad,open_rad2,open_sin2,open_cos2,open_cos,open_tan,open_tan2,open_xsinx,open_xcosx,open_sinxcosx,open_xsinxcosx,open_xsinx2,open_xcosx2,open_sinxcosx2,open_xsinxcosx2,open_xtanx,open_xtanx2,%high,mesa_high,high_amp,high_omega,high_phase,high_offset,high_freq,high_period,high_maxcov,high_sin,high_angle,high_rad,high_rad2,high_sin2,high_cos2,high_cos,high_tan,high_tan2,high_xsinx,high_xcosx,high_sinxcosx,high_xsinxcosx,high_xsinx2,high_xcosx2,high_sinxcosx2,high_xsinxcosx2,high_xtanx,high_xtanx2,%low,mesa_low,low_amp,low_omega,low_phase,low_offset,low_freq,low_period,low_maxcov,low_sin,low_angle,low_rad,low_rad2,low_sin2,low_cos2,low_cos,low_tan,low_tan2,low_xsinx,low_xcosx,low_sinxcosx,low_xsinxcosx,low_xsinx2,low_xcosx2,low_sinxcosx2,low_xsinxcosx2,low_xtanx,low_xtanx2,...,%high_std30,%high_std50,%high_var30,%high_var50,%high_min30,%high_min50,%high_max30,%high_max50,%high_skew30,%high_skew50,%high_kurt30,%high_kurt50,%low_1,%low_2,%low_3,%low_4,%low_5,%low_6,%low_7,%low_8,%low_9,%low_10,%low_11,%low_12,%low_13,%low_14,%low_15,%low_16,%low_17,%low_18,%low_19,%low_20,%low_21,%low_22,%low_23,%low_24,%low_25,%low_26,%low_27,%low_28,%low_29,%low_30,%low_31,%low_32,%low_33,%low_34,%low_35,%low_36,%low_37,%low_38,%low_39,%low_40,%low_41,%low_42,%low_43,%low_44,%low_45,%low_46,%low_47,%low_48,%low_49,%low_50,%low_51,%low_52,%low_53,%low_54,%low_55,%low_56,%low_57,%low_58,%low_59,%low_60,low_sma30,low_std30,low_var30,low_min30,low_max30,low_skew30,low_kurt30,low_sma50,low_std50,low_var50,low_min50,low_max50,low_skew50,low_kurt50,%low_sma30,%low_sma50,%low_std30,%low_std50,%low_var30,%low_var50,%low_min30,%low_min50,%low_max30,%low_max50,%low_skew30,%low_skew50,%low_kurt30,%low_kurt50
8845,17.28,17.28,17.28,17.28,100,2020-08-25 22:48:00,2020-08-25,22,48,1368,RRR,17.30,-0.02,-0.001156,-0.001157,-0.001156,0.490588,0.001894,2.999950,141.151213,-0.000400,0.477457,2.094430,2.221737e+05,-0.001211,8845,26675.705720,3.584091,4.061547,-0.001907,-0.605856,-0.903685,0.473836,1.313140,0.000001,0.001045,0.001095,-0.000001,2.204898e-06,0.000700,0.001156,-1.335851e-06,-0.000548,-0.001518,-0.001156,0.490591,-0.001940,2.802241,-332.993391,-0.000401,0.445991,2.242200,2.440428e+05,0.001482,24452.828207,4.954177,5.400168,0.001098,0.634822,0.239439,-4.054941,-1.217125,-0.000002,-0.000277,0.000355,-4.103498e-07,-0.000001,-0.000734,0.000697,-8.057384e-07,0.004688,0.001407,-0.001156,0.490588,0.002575,2.990564,224.104713,-0.000389,0.475963,2.101003,1.440349e+05,-0.001352,26675.646515,3.524886,4.000850,-0.002340,-0.653000,-0.927438,0.403237,1.159812,0.000002,0.001072,0.001254,-0.000001,0.000003,0.000755,0.001528,-0.000002,-0.000466,-0.001341,...,175.770098,74.255262,1807.314088,326.740421,0.004651,0.025519,-0.026479,-0.026479,9.107055,-35.842614,1.618333,-22.554117,-0.002882,0.000000,0.001732,0.000000,0.011092,-0.006381,-0.000580,-0.007480,0.004624,0.000000,0.000000,-0.003456,-0.001151,0.0,0.0,0.0,0.000000,0.000000,0.000000,0.000000,-0.001149,-0.001148,-0.001147,0.000000,-0.000573,0.000000,0.000000,0.004028,0.000000,0.001152,0.003468,-0.009164,0.015707,0.015957,0.000000,0.004154,0.000000,-0.003548,0.003561,-0.003253,0.000000,0.000888,-0.001183,-0.000591,-0.000591,-0.000295,0.000591,0.001183,0.001184,0.000889,0.000000,-0.000888,0.000296,-0.000296,0.000889,0.000593,-0.000296,0.000000,0.000297,-0.002662,17.355000,0.071715,0.005143,17.13,17.45,-1.093830,1.963110,17.2173,0.218411,0.047703,16.85,17.46,-0.677560,-1.307059,-0.004322,0.003642,239.952318,78.116968,3358.839088,361.239276,0.008757,0.025519,-0.009742,-0.010309,-16.797696,-26.503292,7.802358,-14.220519
8844,17.30,17.30,17.30,17.30,200,2020-08-25 22:44:00,2020-08-25,22,44,1364,RRR,17.44,-0.14,-0.008028,-0.008060,-0.002882,0.489934,0.001881,1.287313,-270.196033,-0.000346,0.204882,4

In [6]:
drop_cols = ['datetime',
            'date',
            'min_num',
            'SYMBOL',
            'prev_close',
            'diff_1',
            'time',
            'decision',
            'open_maxcov',
            'high_maxcov',
            'low_maxcov',
            'close_maxcov',
            'pct_change',
            'D2',
            'open',
            'high',
            'low',
            'close']

df.drop(drop_cols, axis=1, inplace=True)

df['target'] = df['%close'].shift(1)

df.dropna(axis=0, inplace=True)

In [7]:
y = df['target']
X = df.drop('target', axis=1)

In [8]:
y.shape, X.shape

((8782,), (8782, 464))

In [9]:
X.head()

,volume,hour,minute,log_return,%open,mesa_open,open_amp,open_omega,open_phase,open_offset,open_freq,open_period,open_sin,open_angle,open_rad,open_rad2,open_sin2,open_cos2,open_cos,open_tan,open_tan2,open_xsinx,open_xcosx,open_sinxcosx,open_xsinxcosx,open_xsinx2,open_xcosx2,open_sinxcosx2,open_xsinxcosx2,open_xtanx,open_xtanx2,%high,mesa_high,high_amp,high_omega,high_phase,high_offset,high_freq,high_period,high_sin,high_angle,high_rad,high_rad2,high_sin2,high_cos2,high_cos,high_tan,high_tan2,high_xsinx,high_xcosx,high_sinxcosx,high_xsinxcosx,high_xsinx2,high_xcosx2,high_sinxcosx2,high_xsinxcosx2,high_xtanx,high_xtanx2,%low,mesa_low,low_amp,low_omega,low_phase,low_offset,low_freq,low_period,low_sin,low_angle,low_rad,low_rad2,low_sin2,low_cos2,low_cos,low_tan,low_tan2,low_xsinx,low_xcosx,low_sinxcosx,low_xsinxcosx,low_xsinx2,low_xcosx2,low_sinxcosx2,low_xsinxcosx2,low_xtanx,low_xtanx2,%close,mesa_close,close_amp,close_omega,close_phase,close_offset,close_freq,close_period,close_sin,close_angle,close_rad,close_rad2,close_sin2,close_cos2,close_cos,...,%high_std30,%high_std50,%high_var30,%high_var50,%high_min30,%high_min50,%high_max30,%high_max50,%high_skew30,%high_skew50,%high_kurt30,%high_kurt50,%low_1,%low_2,%low_3,%low_4,%low_5,%low_6,%low_7,%low_8,%low_9,%low_10,%low_11,%low_12,%low_13,%low_14,%low_15,%low_16,%low_17,%low_18,%low_19,%low_20,%low_21,%low_22,%low_23,%low_24,%low_25,%low_26,%low_27,%low_28,%low_29,%low_30,%low_31,%low_32,%low_33,%low_34,%low_35,%low_36,%low_37,%low_38,%low_39,%low_40,%low_41,%low_42,%low_43,%low_44,%low_45,%low_46,%low_47,%low_48,%low_49,%low_50,%low_51,%low_52,%low_53,%low_54,%low_55,%low_56,%low_57,%low_58,%low_59,%low_60,low_sma30,low_std30,low_var30,low_min30,low_max30,low_skew30,low_kurt30,low_sma50,low_std50,low_var50,low_min50,low_max50,low_skew50,low_kurt50,%low_sma30,%low_sma50,%low_std30,%low_std50,%low_var30,%low_var50,%low_min30,%low_min50,%low_max30,%low_max50,%low_skew30,%low_skew50,%low_kurt30,%low_kurt50
8844,200,22,44,-0.008060,-0.002882,0.489934,0.001881,1.287313,-270.196033,-0.000346,0.204882,4.880852,-0.000633,11114.801657,6.130034,6.334916,-0.000249,0.998662,0.988295,-0.154360,0.051777,0.000002,-0.002848,-0.000626,0.000002,7.176933e-07,-0.002878,-0.000249,7.167332e-07,0.000445,-0.000149,-0.008028,0.488493,-0.002108,2.808354,-387.061715,-0.000335,0.446963,2.237320,-0.002103,24450.020219,2.146189,2.593152,-0.001434,-0.853339,-0.544164,-1.541776,-0.610962,0.000017,0.004368,0.001145,-0.000009,0.000012,0.006850,0.001223,-0.000010,0.012377,0.004905,-0.002882,0.489934,0.002457,3.001820,124.667322,-0.000327,0.477755,2.093125,0.001151,26672.767225,0.645596,1.123351,0.001888,0.432664,0.798741,0.753279,2.083731,-0.000003,-0.002302,0.000919,-0.000003,-0.000005,-0.001247,0.000817,-0.000002,-0.002171,-0.006005,-0.008028,0.488493,-0.002719,2.838034,-649.214703,-0.000325,0.451687,2.213922,-0.001990,24450.356426,2.482395,2.934083,-0.000885,-0.978547,-0.790484,...,179.099649,72.732683,1873.906572,313.249046,0.005814,0.026706,-0.025352,-0.025352,9.016392,-40.993553,1.473616,-19.012349,0.000000,0.001732,0.000000,0.011092,-0.006381,-0.000580,-0.007480,0.004624,0.000000,0.000000,-0.003456,-0.001151,0.0,0.0,0.0,0.000000,0.000000,0.000000,0.000000,-0.001149,-0.001148,-0.001147,0.000000,-0.000573,0.000000,0.000000,0.004028,0.000000,0.001152,0.003468,-0.009164,0.015707,0.015957,0.000000,0.004154,0.000000,-0.003548,0.003561,-0.003253,0.000000,0.000888,-0.001183,-0.000591,-0.000591,-0.000295,0.000591,0.001183,0.001184,0.000889,0.000000,-0.000888,0.000296,-0.000296,0.000889,0.000593,-0.000296,0.000000,0.000297,-0.002662,-0.000887,17.358333,0.070422,0.004959,17.13,17.45,-1.261216,2.567249,17.2094,0.223188,0.049813,16.85,17.46,-0.594950,-1.450096,-0.003361,0.005265,244.663420,76.513117,3487.469116,346.299609,0.009924,0.026706,-0.008596,-0.009164,-14.716923,-30.078059,5.738731,-12.930245
8843,25,22,36,0.005174,0.000000,0.496402,0.004198,3.106833,-802.280898,-0.000187,0.494468,2.022376,-0.002822,26671.442713,

In [10]:
from sklearn.preprocessing import StandardScaler

ss = StandardScaler()
ss.fit(X)
X = pd.DataFrame(ss.transform(X), index=X.index, columns=X.columns)
X.head()

,volume,hour,minute,log_return,%open,mesa_open,open_amp,open_omega,open_phase,open_offset,open_freq,open_period,open_sin,open_angle,open_rad,open_rad2,open_sin2,open_cos2,open_cos,open_tan,open_tan2,open_xsinx,open_xcosx,open_sinxcosx,open_xsinxcosx,open_xsinx2,open_xcosx2,open_sinxcosx2,open_xsinxcosx2,open_xtanx,open_xtanx2,%high,mesa_high,high_amp,high_omega,high_phase,high_offset,high_freq,high_period,high_sin,high_angle,high_rad,high_rad2,high_sin2,high_cos2,high_cos,high_tan,high_tan2,high_xsinx,high_xcosx,high_sinxcosx,high_xsinxcosx,high_xsinx2,high_xcosx2,high_sinxcosx2,high_xsinxcosx2,high_xtanx,high_xtanx2,%low,mesa_low,low_amp,low_omega,low_phase,low_offset,low_freq,low_period,low_sin,low_angle,low_rad,low_rad2,low_sin2,low_cos2,low_cos,low_tan,low_tan2,low_xsinx,low_xcosx,low_sinxcosx,low_xsinxcosx,low_xsinx2,low_xcosx2,low_sinxcosx2,low_xsinxcosx2,low_xtanx,low_xtanx2,%close,mesa_close,close_amp,close_omega,close_phase,close_offset,close_freq,close_period,close_sin,close_angle,close_rad,close_rad2,close_sin2,close_cos2,close_cos,...,%high_std30,%high_std50,%high_var30,%high_var50,%high_min30,%high_min50,%high_max30,%high_max50,%high_skew30,%high_skew50,%high_kurt30,%high_kurt50,%low_1,%low_2,%low_3,%low_4,%low_5,%low_6,%low_7,%low_8,%low_9,%low_10,%low_11,%low_12,%low_13,%low_14,%low_15,%low_16,%low_17,%low_18,%low_19,%low_20,%low_21,%low_22,%low_23,%low_24,%low_25,%low_26,%low_27,%low_28,%low_29,%low_30,%low_31,%low_32,%low_33,%low_34,%low_35,%low_36,%low_37,%low_38,%low_39,%low_40,%low_41,%low_42,%low_43,%low_44,%low_45,%low_46,%low_47,%low_48,%low_49,%low_50,%low_51,%low_52,%low_53,%low_54,%low_55,%low_56,%low_57,%low_58,%low_59,%low_60,low_sma30,low_std30,low_var30,low_min30,low_max30,low_skew30,low_kurt30,low_sma50,low_std50,low_var50,low_min50,low_max50,low_skew50,low_kurt50,%low_sma30,%low_sma50,%low_std30,%low_std50,%low_var30,%low_var50,%low_min30,%low_min50,%low_max30,%low_max50,%low_skew30,%low_skew50,%low_kurt30,%low_kurt50
8844,-0.371454,2.695687,0.783583,-2.679836,-1.036307,-0.528959,-0.009136,-0.334705,-0.768433,-0.022016,-0.334705,-0.107162,-0.449236,0.731524,1.584130,1.549223,-0.022872,1.349414,1.333115,-0.012039,0.003295,-0.073074,-1.640534,-0.680856,0.285129,0.040519,-1.602510,-0.014112,-0.010287,0.020964,-0.000559,-2.917306,-1.182598,-0.022318,1.591103,-1.104468,-0.018639,1.591103,-0.141867,-1.417974,3.012835,-0.560842,-0.441713,-0.040839,-1.181375,-0.758717,-0.013949,0.011394,1.197469,2.702885,1.284675,-1.417391,0.066043,4.162280,0.068464,-0.027905,0.051880,0.018395,-1.025586,-0.529158,-0.001576,1.732746,0.334523,-0.007925,1.732746,-0.136428,0.696767,3.606474,-1.378238,-1.243917,0.044286,0.596044,1.096697,0.015930,0.006081,-0.417023,-1.461920,0.999761,-0.433018,0.002434,-0.741193,0.020957,-0.025749,0.005927,-0.039050,-2.668346,-1.133217,-0.021505,1.347265,-1.846382,-0.008933,1.347265,-0.125014,-1.277994,2.829309,-0.388371,-0.285432,-0.037508,-1.349284,-1.083758,...,-0.763213,-1.105646,-0.547496,-0.629264,-0.239363,1.144408,-2.022224,-1.363797,0.000942,-0.012725,0.013393,0.004579,-0.021652,0.581990,-0.021511,3.833294,-2.240038,-0.222644,-2.623674,1.588011,-0.021619,-0.021172,-1.224280,-0.421942,-0.021469,-0.02155,-0.021986,-0.021726,-0.021823,-0.021787,-0.021680,-0.421871,-0.421234,-0.420877,-0.022027,-0.221156,-0.021951,-0.021765,1.379473,-0.021411,0.379173,1.184881,-3.205985,5.443785,5.538107,-0.019803,1.429198,-0.019648,-1.257894,1.222520,-1.155421,-0.020288,0.289716,-0.432728,-0.226298,-0.226130,-0.122983,0.186256,0.392649,0.393687,0.290220,-0.020180,-0.329490,0.083332,-0.123083,0.290034,0.186808,-0.123047,-0.019665,0.083709,-0.947087,-0.325726,1.421577,0.225806,-0.073664,1.368263,1.408323,-1.675138,1.336554,1.359033,2.355845,1.985910,1.263317,1.39659,-0.727306,-0.606180,-0.568782,0.396724,-0.522160,-1.078693,-0.499889,-0.619833,0.133000,1.063599,-0.198454,-0.014509,-0.013152,-0.001022,0.015571,0.004299
8843,-0.383270,2.695687,0.322643,1.687568,-0.022258,2.376909,0.001574,1.759803,-2.30399

In [11]:
X = sm.add_constant(X)
model = sm.OLS(y,X)
results = model.fit()

In [12]:
results.summary()

<class 'statsmodels.iolib.summary.Summary'>
"""
                            OLS Regression Results                            
==============================================================================
Dep. Variable:                 target   R-squared:                       0.184
Model:                            OLS   Adj. R-squared:                  0.140
Method:                 Least Squares   F-statistic:                     4.126
Date:                Mon, 07 Sep 2020   Prob (F-statistic):          3.05e-149
Time:                        19:51:42   Log-Likelihood:                 39351.
No. Observations:                8782   AIC:                        -7.779e+04
Df Residuals:                    8325   BIC:                        -7.455e+04
Df Model:                         456                                         
Covariance Type:            nonrobust                                         
====================================================================================
                       coef    std err          t      P>|t|      [0.025      0.975]
------------------------------------------------------------------------------------
const             6.087e-05      3e-05      2.027      0.043    2.01e-06       0.000
volume            8.395e-05   3.25e-05      2.580      0.010    2.02e-05       0.000
hour             -6.646e-05   4.68e-05     -1.420      0.156      -0.000    2.53e-05
minute           -3.513e-05   3.31e-05     -1.062      0.288     -0.0001    2.97e-05
log_return          -0.0639      0.010     -6.226      0.000      -0.084      -0.044
%open               -0.0141      0.005     -2.899      0.004      -0.024      -0.005
mesa_open            0.0331      0.006      5.983      0.000       0.022       0.044
open_amp          5.049e-05   3.97e-05      1.271      0.204   -2.74e-05       0.000
open_omega        4.575e-05   3.82e-05      1.197      0.232   -2.92e-05       0.000
open_phase       -3.353e-05   3.38e-05     -0.991      0.322   -9.98e-05    3.28e-05
open_offset       1.608e-05   3.99e-05      0.403      0.687   -6.21e-05    9.43e-05
open_freq         4.575e-05   3.82e-05      1.197      0.232   -2.92e-05       0.000
open_period      -1.346e-06   4.14e-05     -0.033      0.974   -8.24e-05    7.98e-05
open_sin         -6.619e-06   4.79e-05     -0.138      0.890      -0.000    8.72e-05
open_angle       -8.941e-05   9.82e-05     -0.910      0.363      -0.000       0.000
open_rad          2.253e-05   2.14e-05      1.054      0.292   -1.94e-05    6.44e-05
open_rad2         2.579e-05   2.11e-05      1.223      0.222   -1.56e-05    6.71e-05
open_sin2         1.815e-05   4.57e-05      0.398      0.691   -7.13e-05       0.000
open_cos2           -0.0003      0.000     -1.860      0.063      -0.001    1.65e-05
open_cos             0.0002      0.000      1.376      0.169   -9.43e-05       0.001
open_tan          4.375e-05   3.78e-05      1.158      0.247   -3.03e-05       0.000
open_tan2        -2.694e-06   3.68e-05     -0.073      0.942   -7.49e-05    6.95e-05
open_xsinx           0.0002   6.71e-05      2.714      0.007    5.05e-05       0.000
open_xcosx       -3.527e-05      0.000     -0.349      0.727      -0.000       0.000
open_sinxcosx    -9.018e-05   4.14e-05     -2.178      0.029      -0.000   -9.01e-06
open_xsinxcosx   -9.241e-05   4.45e-05     -2.078      0.038      -0.000   -5.23e-06
open_xsinx2         -0.0004   6.84e-05     -5.282      0.000      -0.000      -0.000
open_xcosx2          0.0001    9.9e-05      1.383      0.167   -5.72e-05       0.000
open_sinxcosx2   -3.309e-05    4.8e-05     -0.689      0.491      -0.000     6.1e-05
open_xsinxcosx2     -0.0003   7.12e-05     -4.727      0.000      -0.000      -0.000
open_xtanx        5.114e-05   3.85e-05      1.328      0.184   -2.44e-05       0.000
open_xtanx2      -8.855e-05   3.74e-05     -2.368      0.018      -0.000   -1.52e-05
%high                0.0067      0.005      1.352      0.176      -0.003       0.016
mesa_high           -0.

In [37]:
results.summary()

<class 'statsmodels.iolib.summary.Summary'>
"""
                            OLS Regression Results                            
==============================================================================
Dep. Variable:                 target   R-squared:                       0.190
Model:                            OLS   Adj. R-squared:                  0.145
Method:                 Least Squares   F-statistic:                     4.231
Date:                Mon, 07 Sep 2020   Prob (F-statistic):          7.81e-157
Time:                        18:48:37   Log-Likelihood:                 39380.
No. Observations:                8782   AIC:                        -7.784e+04
Df Residuals:                    8321   BIC:                        -7.457e+04
Df Model:                         460                                         
Covariance Type:            nonrobust                                         
====================================================================================
                       coef    std err          t      P>|t|      [0.025      0.975]
------------------------------------------------------------------------------------
const               -6.5229      1.133     -5.757      0.000      -8.744      -4.302
open                 0.0156      0.017      0.917      0.359      -0.018       0.049
high                -0.0752      0.018     -4.184      0.000      -0.110      -0.040
low                  0.0445      0.018      2.510      0.012       0.010       0.079
close                0.0100      0.018      0.567      0.571      -0.025       0.045
volume            6.103e-09   2.19e-09      2.785      0.005    1.81e-09    1.04e-08
hour              -3.26e-05    2.2e-05     -1.480      0.139   -7.58e-05    1.06e-05
minute            -2.07e-06    1.9e-06     -1.089      0.276   -5.79e-06    1.65e-06
log_return         -20.8773      3.387     -6.164      0.000     -27.517     -14.237
%open               -4.4601      1.708     -2.611      0.009      -7.808      -1.112
mesa_open           14.6226      2.478      5.901      0.000       9.765      19.480
open_amp             0.0002      0.000      1.315      0.189      -0.000       0.001
open_omega           0.0001   8.47e-05      1.331      0.183   -5.33e-05       0.000
open_phase       -8.053e-08   9.74e-08     -0.827      0.408   -2.71e-07     1.1e-07
open_offset       8.381e-05      0.000      0.441      0.659      -0.000       0.000
open_freq         1.583e-05   1.35e-05      1.173      0.241   -1.06e-05    4.23e-05
open_period      -1.676e-09   3.42e-07     -0.005      0.996   -6.73e-07    6.69e-07
open_sin             0.0036      0.031      0.117      0.907      -0.057       0.064
open_angle       -1.824e-08    1.7e-08     -1.074      0.283   -5.15e-08    1.51e-08
open_rad          2.872e-06   1.36e-05      0.212      0.832   -2.37e-05    2.94e-05
open_rad2         2.325e-05   1.29e-05      1.803      0.071   -2.03e-06    4.85e-05
open_sin2            0.0011      0.002      0.641      0.522      -0.002       0.005
open_cos2           -0.0004      0.000     -1.876      0.061      -0.001     1.9e-05
open_cos             0.0003      0.000      1.403      0.161      -0.000       0.001
open_tan          3.228e-07   2.77e-07      1.167      0.243    -2.2e-07    8.65e-07
open_tan2         7.344e-09   4.82e-07      0.015      0.988   -9.38e-07    9.53e-07
open_xsinx          13.1883      5.763      2.288      0.022       1.890      24.486
open_xcosx          -0.0096      0.058     -0.167      0.868      -0.123       0.104
open_sinxcosx       -0.0967      0.045     -2.145      0.032      -0.185      -0.008
open_xsinxcosx     -13.6373      6.854     -1.990      0.047     -27.073      -0.202
open_xsinx2         -1.2843      0.243     -5.291      0.000      -1.760      -0.809
open_xcosx2          0.0697      0.055      1.262      0.207      -0.039       0.178
open_sinxcosx2      -0.0017      0.002     -0.814      0.416      -0.006       0.002
open_xsinxcosx2     -1.

In [13]:
pmask = results.pvalues < 0.05

pcols = X.columns[pmask]

X2 = X[pcols]

X2.shape

(8782, 180)

In [14]:
model2 = sm.OLS(y,X2)
results2 = model2.fit()

In [15]:
results2.summary()

<class 'statsmodels.iolib.summary.Summary'>
"""
                            OLS Regression Results                            
==============================================================================
Dep. Variable:                 target   R-squared:                       0.111
Model:                            OLS   Adj. R-squared:                  0.093
Method:                 Least Squares   F-statistic:                     6.024
Date:                Mon, 07 Sep 2020   Prob (F-statistic):          7.19e-119
Time:                        19:52:35   Log-Likelihood:                 38975.
No. Observations:                8782   AIC:                        -7.759e+04
Df Residuals:                    8602   BIC:                        -7.632e+04
Df Model:                         179                                         
Covariance Type:            nonrobust                                         
====================================================================================
                       coef    std err          t      P>|t|      [0.025      0.975]
------------------------------------------------------------------------------------
const             6.087e-05   3.08e-05      1.974      0.048    4.32e-07       0.000
volume            9.274e-05    3.2e-05      2.902      0.004    3.01e-05       0.000
log_return          -0.0809      0.007    -11.406      0.000      -0.095      -0.067
%open                0.0004      0.000      3.685      0.000       0.000       0.001
mesa_open        -5.944e-05      0.000     -0.211      0.833      -0.001       0.000
open_xsinx       -6.413e-05    3.7e-05     -1.732      0.083      -0.000    8.47e-06
open_sinxcosx    -5.688e-05   3.32e-05     -1.712      0.087      -0.000    8.24e-06
open_xsinxcosx    -4.89e-05   3.91e-05     -1.250      0.211      -0.000    2.78e-05
open_xsinx2         -0.0004   6.33e-05     -6.418      0.000      -0.001      -0.000
open_xsinxcosx2     -0.0003   6.81e-05     -4.173      0.000      -0.000      -0.000
open_xtanx2      -9.974e-05   3.15e-05     -3.167      0.002      -0.000    -3.8e-05
high_phase       -7.611e-05   3.13e-05     -2.431      0.015      -0.000   -1.47e-05
high_sin            -0.0001   4.21e-05     -3.259      0.001      -0.000   -5.46e-05
high_angle        2.666e-05   3.14e-05      0.850      0.395   -3.48e-05    8.81e-05
high_sin2           -0.0002   4.92e-05     -3.090      0.002      -0.000   -5.56e-05
high_cos2           -0.0002      0.000     -1.826      0.068      -0.000    1.75e-05
high_cos             0.0002      0.000      1.367      0.172   -7.76e-05       0.000
high_xcosx           0.0001   9.05e-05      1.503      0.133   -4.14e-05       0.000
high_xsinxcosx       0.0003   4.52e-05      6.550      0.000       0.000       0.000
high_xsinx2          0.0002   4.29e-05      3.965      0.000     8.6e-05       0.000
high_xcosx2         -0.0001   8.99e-05     -1.598      0.110      -0.000    3.26e-05
high_xsinxcosx2      0.0003   5.38e-05      5.864      0.000       0.000       0.000
high_xtanx2      -6.308e-05   3.16e-05     -1.994      0.046      -0.000   -1.07e-06
mesa_low            -0.0002      0.000     -1.158      0.247      -0.000       0.000
low_sin              0.0002   4.23e-05      5.532      0.000       0.000       0.000
low_tan2            -0.0002   4.54e-05     -3.758      0.000      -0.000   -8.17e-05
low_xcosx            0.0002   8.94e-05      2.536      0.011    5.15e-05       0.000
low_xsinx2           0.0002   6.62e-05      3.743      0.000       0.000       0.000
low_xcosx2          -0.0002   8.98e-05     -1.839      0.066      -0.000    1.09e-05
low_sinxcosx2     3.647e-05   3.65e-05      0.998      0.318   -3.51e-05       0.000
low_xsinxcosx2       0.0004   7.27e-05      5.865      0.000       0.000       0.001
low_xtanx2           0.0002   4.57e-05      4.087      0.000    9.71e-05       0.000
%close               0.0631      0.005     11.789      0.000       0.053       0.074
mesa_close           0.

In [16]:
pmask2 = results2.pvalues < 0.05

pcols2 = X2.columns[pmask2]

X3 = X2[pcols2]

X3.shape

(8782, 84)

In [17]:
model3 = sm.OLS(y,X3)
results3 = model3.fit()

In [18]:
results3.summary()

<class 'statsmodels.iolib.summary.Summary'>
"""
                            OLS Regression Results                            
==============================================================================
Dep. Variable:                 target   R-squared:                       0.086
Model:                            OLS   Adj. R-squared:                  0.077
Method:                 Least Squares   F-statistic:                     9.875
Date:                Mon, 07 Sep 2020   Prob (F-statistic):          3.58e-115
Time:                        19:52:38   Log-Likelihood:                 38852.
No. Observations:                8782   AIC:                        -7.754e+04
Df Residuals:                    8698   BIC:                        -7.694e+04
Df Model:                          83                                         
Covariance Type:            nonrobust                                         
====================================================================================
                       coef    std err          t      P>|t|      [0.025      0.975]
------------------------------------------------------------------------------------
const             6.087e-05   3.11e-05      1.958      0.050   -8.21e-08       0.000
volume            9.932e-05   3.13e-05      3.170      0.002    3.79e-05       0.000
log_return          -0.0854      0.007    -12.861      0.000      -0.098      -0.072
%open             9.441e-05   4.68e-05      2.019      0.044    2.75e-06       0.000
open_xsinx2         -0.0004   6.17e-05     -6.349      0.000      -0.001      -0.000
open_xsinxcosx2     -0.0003   6.33e-05     -4.124      0.000      -0.000      -0.000
open_xtanx2      -9.789e-05   3.13e-05     -3.126      0.002      -0.000   -3.65e-05
high_phase       -7.225e-05   3.13e-05     -2.308      0.021      -0.000   -1.09e-05
high_sin         -6.967e-05   3.93e-05     -1.773      0.076      -0.000    7.38e-06
high_sin2           -0.0001   4.86e-05     -2.748      0.006      -0.000   -3.83e-05
high_xsinxcosx       0.0003    4.3e-05      6.494      0.000       0.000       0.000
high_xsinx2          0.0002    4.1e-05      4.016      0.000    8.43e-05       0.000
high_xsinxcosx2      0.0003   5.33e-05      5.719      0.000       0.000       0.000
high_xtanx2      -5.077e-05   3.17e-05     -1.604      0.109      -0.000    1.13e-05
low_sin              0.0002   3.87e-05      6.269      0.000       0.000       0.000
low_tan2            -0.0002   4.56e-05     -3.716      0.000      -0.000   -8.01e-05
low_xcosx         5.901e-05   3.33e-05      1.770      0.077   -6.35e-06       0.000
low_xsinx2           0.0003   6.53e-05      3.911      0.000       0.000       0.000
low_xsinxcosx2       0.0004    7.2e-05      6.011      0.000       0.000       0.001
low_xtanx2           0.0002   4.58e-05      3.984      0.000    9.27e-05       0.000
%close               0.0653      0.005     13.439      0.000       0.056       0.075
mesa_close           0.0284      0.003      9.012      0.000       0.022       0.035
close_sin2           0.0001   4.38e-05      3.022      0.003    4.65e-05       0.000
close_tan            0.0086      0.002      3.760      0.000       0.004       0.013
close_xcosx         -0.0005    7.8e-05     -5.880      0.000      -0.001      -0.000
close_xsinxcosx     -0.0002   4.07e-05     -4.332      0.000      -0.000   -9.66e-05
close_xcosx2         0.0004   7.85e-05      4.967      0.000       0.000       0.001
close_xsinxcosx2    -0.0003   6.29e-05     -5.153      0.000      -0.000      -0.000
close_xtanx          0.0086      0.002      3.757      0.000       0.004       0.013
%open_20          1.099e-05   4.67e-05      0.235      0.814   -8.05e-05       0.000
%open_21            -0.0002   7.98e-05     -2.773      0.006      -0.000   -6.48e-05
%open_22         -4.438e-05   9.21e-05     -0.482      0.630      -0.000       0.000
%open_23            -0.0001   9.38e-05     -1.257      0.209      -0.000    6.59e-05
%open_24            -0.

In [19]:
pmask3 = results3.pvalues < 0.05

pcols3 = X3.columns[pmask3]

X4 = X3[pcols3]

X4.shape

(8782, 54)

In [20]:
model4 = sm.OLS(y,X4)
results4 = model4.fit()
results4.summary()

<class 'statsmodels.iolib.summary.Summary'>
"""
                                 OLS Regression Results                                
=======================================================================================
Dep. Variable:                 target   R-squared (uncentered):                   0.082
Model:                            OLS   Adj. R-squared (uncentered):              0.076
Method:                 Least Squares   F-statistic:                              14.47
Date:                Mon, 07 Sep 2020   Prob (F-statistic):                   2.46e-123
Time:                        19:52:41   Log-Likelihood:                          38831.
No. Observations:                8782   AIC:                                 -7.755e+04
Df Residuals:                    8728   BIC:                                 -7.717e+04
Df Model:                          54                                                  
Covariance Type:            nonrobust                                                  
====================================================================================
                       coef    std err          t      P>|t|      [0.025      0.975]
------------------------------------------------------------------------------------
volume            9.797e-05   3.13e-05      3.134      0.002    3.67e-05       0.000
log_return          -0.0857      0.007    -12.970      0.000      -0.099      -0.073
%open             5.995e-05   4.54e-05      1.320      0.187   -2.91e-05       0.000
open_xsinx2         -0.0004   6.15e-05     -6.467      0.000      -0.001      -0.000
open_xsinxcosx2     -0.0003   6.32e-05     -4.068      0.000      -0.000      -0.000
open_xtanx2      -9.661e-05   3.12e-05     -3.094      0.002      -0.000   -3.54e-05
high_phase       -7.415e-05   3.13e-05     -2.371      0.018      -0.000   -1.28e-05
high_sin2           -0.0001   4.82e-05     -2.726      0.006      -0.000    -3.7e-05
high_xsinxcosx       0.0003   4.28e-05      6.428      0.000       0.000       0.000
high_xsinx2          0.0002    4.1e-05      4.032      0.000    8.49e-05       0.000
high_xsinxcosx2      0.0003   5.32e-05      5.782      0.000       0.000       0.000
low_sin              0.0002   3.77e-05      6.300      0.000       0.000       0.000
low_tan2            -0.0002   4.56e-05     -3.666      0.000      -0.000   -7.77e-05
low_xsinx2           0.0003   6.51e-05      3.871      0.000       0.000       0.000
low_xsinxcosx2       0.0004   7.18e-05      6.024      0.000       0.000       0.001
low_xtanx2           0.0002   4.57e-05      3.931      0.000       9e-05       0.000
%close               0.0653      0.005     13.508      0.000       0.056       0.075
mesa_close           0.0287      0.003      9.172      0.000       0.023       0.035
close_sin2           0.0001   4.36e-05      3.307      0.001    5.86e-05       0.000
close_tan            0.0086      0.002      3.825      0.000       0.004       0.013
close_xcosx         -0.0004   7.74e-05     -5.635      0.000      -0.001      -0.000
close_xsinxcosx     -0.0002   4.05e-05     -4.434      0.000      -0.000      -0.000
close_xcosx2         0.0004   7.83e-05      4.920      0.000       0.000       0.001
close_xsinxcosx2    -0.0003   6.26e-05     -5.417      0.000      -0.000      -0.000
close_xtanx          0.0086      0.002      3.822      0.000       0.004       0.013
%open_21            -0.0002   5.23e-05     -3.905      0.000      -0.000      -0.000
open_kurt50         -0.0002   4.45e-05     -4.583      0.000      -0.000      -0.000
%open_sma30         -0.0003   4.95e-05     -5.736      0.000      -0.000      -0.000
%close_1            -0.0098      0.001     -9.319      0.000      -0.012      -0.008
%close_2            -0.0096      0.001     -9.143      0.000      -0.012      -0.008
%close_3            -0.0200      0.002     -9.186      0.000      -0.024      -0.016
%close_21            0.0002   5.82e-05      3.377      0.001    8.25e-05       0.000
%close_22

In [21]:
ys = y.copy()
ys

8844   -0.001156
8843   -0.008028
8842    0.005187
8841    0.001732
8840    0.000000
          ...   
65      0.000000
64     -0.000960
63      0.000000
62     -0.002871
61      0.003842
Name: target, Length: 8782, dtype: float64

In [22]:
ys[ys > 0] = 1
ys[ys < 0] = 0

In [23]:
ys

8844    0.0
8843    0.0
8842    1.0
8841    1.0
8840    0.0
       ... 
65      0.0
64      0.0
63      0.0
62      0.0
61      1.0
Name: target, Length: 8782, dtype: float64

In [24]:
X3.drop('const', axis=1, inplace=True)

/opt/conda/lib/python3.7/site-packages/pandas/core/frame.py:4167: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  errors=errors,


In [25]:
from sklearn.linear_model import LogisticRegression

lr = LogisticRegression()

lr.fit(X3, ys)
lr.score(X3, ys)

/opt/conda/lib/python3.7/site-packages/sklearn/linear_model/_logistic.py:764: ConvergenceWarning: lbfgs failed to converge (status=1):
STOP: TOTAL NO. of ITERATIONS REACHED LIMIT.

Increase the number of iterations (max_iter) or scale the data as shown in:
    https://scikit-learn.org/stable/modules/preprocessing.html
Please also refer to the documentation for alternative solver options:
    https://scikit-learn.org/stable/modules/linear_model.html#logistic-regression
  extra_warning_msg=_LOGISTIC_SOLVER_CONVERGENCE_MSG)


0.571965383739467

In [26]:
ys.value_counts() / ys.shape[0]

0.0    0.560351
1.0    0.439649
Name: target, dtype: float64

In [27]:
from sklearn.preprocessing import PolynomialFeatures

poly = PolynomialFeatures(include_bias=True)
poly.fit(X3)
X3_poly = poly.transform(X3)
X3_poly = pd.DataFrame(X3_poly, index=X3.index, columns=poly.get_feature_names(X3.columns))

In [28]:
X3_poly

,1,volume,log_return,%open,open_xsinx2,open_xsinxcosx2,open_xtanx2,high_phase,high_sin,high_sin2,high_xsinxcosx,high_xsinx2,high_xsinxcosx2,high_xtanx2,low_sin,low_tan2,low_xcosx,low_xsinx2,low_xsinxcosx2,low_xtanx2,%close,mesa_close,close_sin2,close_tan,close_xcosx,close_xsinxcosx,close_xcosx2,close_xsinxcosx2,close_xtanx,%open_20,%open_21,%open_22,%open_23,%open_24,%open_41,%open_42,%open_43,open_kurt50,%open_sma30,%open_sma50,%close_1,%close_2,%close_3,%close_21,%close_22,%close_23,%close_24,%close_25,%close_26,%close_27,%close_28,%close_29,%close_30,%close_31,close_kurt50,%high_13,%high_14,%high_15,%high_16,%high_18,%high_19,%high_21,%high_22,%high_23,%high_24,%high_25,%high_26,%high_27,%high_28,%high_29,%high_30,%high_31,%low_4,%low_6,%low_7,%low_8,%low_9,%low_25,%low_26,%low_27,%low_28,%low_29,%low_30,%low_31,volume^2,volume log_return,volume %open,volume open_xsinx2,volume open_xsinxcosx2,volume open_xtanx2,volume high_phase,volume high_sin,volume high_sin2,volume high_xsinxcosx,volume high_xsinx2,volume high_xsinxcosx2,volume high_xtanx2,volume low_sin,volume low_tan2,volume low_xcosx,...,%high_30 %low_8,%high_30 %low_9,%high_30 %low_25,%high_30 %low_26,%high_30 %low_27,%high_30 %low_28,%high_30 %low_29,%high_30 %low_30,%high_30 %low_31,%high_31^2,%high_31 %low_4,%high_31 %low_6,%high_31 %low_7,%high_31 %low_8,%high_31 %low_9,%high_31 %low_25,%high_31 %low_26,%high_31 %low_27,%high_31 %low_28,%high_31 %low_29,%high_31 %low_30,%high_31 %low_31,%low_4^2,%low_4 %low_6,%low_4 %low_7,%low_4 %low_8,%low_4 %low_9,%low_4 %low_25,%low_4 %low_26,%low_4 %low_27,%low_4 %low_28,%low_4 %low_29,%low_4 %low_30,%low_4 %low_31,%low_6^2,%low_6 %low_7,%low_6 %low_8,%low_6 %low_9,%low_6 %low_25,%low_6 %low_26,%low_6 %low_27,%low_6 %low_28,%low_6 %low_29,%low_6 %low_30,%low_6 %low_31,%low_7^2,%low_7 %low_8,%low_7 %low_9,%low_7 %low_25,%low_7 %low_26,%low_7 %low_27,%low_7 %low_28,%low_7 %low_29,%low_7 %low_30,%low_7 %low_31,%low_8^2,%low_8 %low_9,%low_8 %low_25,%low_8 %low_26,%low_8 %low_27,%low_8 %low_28,%low_8 %low_29,%low_8 %low_30,%low_8 %low_31,%low_9^2,%low_9 %low_25,%low_9 %low_26,%low_9 %low_27,%low_9 %low_28,%low_9 %low_29,%low_9 %low_30,%low_9 %low_31,%low_25^2,%low_25 %low_26,%low_25 %low_27,%low_25 %low_28,%low_25 %low_29,%low_25 %low_30,%low_25 %low_31,%low_26^2,%low_26 %low_27,%low_26 %low_28,%low_26 %low_29,%low_26 %low_30,%low_26 %low_31,%low_27^2,%low_27 %low_28,%low_27 %low_29,%low_27 %low_30,%low_27 %low_31,%low_28^2,%low_28 %low_29,%low_28 %low_30,%low_28 %low_31,%low_29^2,%low_29 %low_30,%low_29 %low_31,%low_30^2,%low_30 %low_31,%low_31^2
8844,1.0,-0.371454,-2.679836,-1.036307,0.040519,-0.010287,-0.000559,-1.104468,-1.417974,-0.040839,-1.417391,0.066043,-0.027905,0.018395,0.696767,0.006081,-1.461920,0.002434,-0.025749,-0.039050,-2.668346,-1.133217,-0.037508,0.010711,3.733733,-1.845919,4.565064,-0.035978,-0.010237,-0.829685,-0.626843,0.179515,-0.223464,-0.022001,-0.228976,-0.228691,-0.228551,-0.713726,-0.598311,0.371711,1.687521,0.551317,-0.020541,-0.399335,-0.587790,0.168026,-0.209638,-0.020847,-0.021053,-5.591744,7.007374,0.360872,-0.400752,-1.536245,-0.431565,-0.022125,-0.022121,-0.022501,-0.022474,-0.022301,-0.022344,-0.642285,-0.022627,-0.022402,-0.022452,-0.022499,-0.022762,-6.114807,7.661975,0.395318,-1.061765,-1.472028,3.833294,-0.222644,-2.623674,1.588011,-0.021619,-0.021951,-0.021765,1.379473,-0.021411,0.379173,1.184881,-3.205985,0.137978,0.995435,0.384940,-0.015051,0.003821,0.000208,0.410259,0.526712,0.015170,0.526495,-0.024532,0.010365,-0.006833,-0.258816,-0.002259,0.543036,...,-1.686095,0.022954,0.023307,0.023109,-1.464676,0.022733,-0.402593,-1.258065,3.404003,2.166868,-5.642718,0.327739,3.862123,-2.337597,0.031824,0.032313,0.032038,-2.030623,0.031517,-0.558153,-1.744178,4.719301,14.694141,-0.853460,-10.057313,6.087312,-0.082871,-0.084146,-0.083431,5.287924,-0.082074,1.453481,4.541996,-12.289481,0.049570,0.584146,-0.353561,0.004813,0.004887,0.004846,-0.307131,0.004767,-0.084421,-0.263807,0.713794,6.883665,-4.

In [29]:
model5 = sm.OLS(y, X3_poly)
results5 = model5.fit()
results5.summary()

<class 'statsmodels.iolib.summary.Summary'>
"""
                            OLS Regression Results                            
==============================================================================
Dep. Variable:                 target   R-squared:                       0.790
Model:                            OLS   Adj. R-squared:                  0.647
Method:                 Least Squares   F-statistic:                     5.519
Date:                Mon, 07 Sep 2020   Prob (F-statistic):               0.00
Time:                        19:54:27   Log-Likelihood:                 45319.
No. Observations:                8782   AIC:                        -8.351e+04
Df Residuals:                    5216   BIC:                        -5.826e+04
Df Model:                        3565                                         
Covariance Type:            nonrobust                                         
====================================================================================================
                                       coef    std err          t      P>|t|      [0.025      0.975]
----------------------------------------------------------------------------------------------------
1                                 1.354e+04    1.7e+04      0.798      0.425   -1.97e+04    4.68e+04
volume                               0.0007      0.001      0.551      0.582      -0.002       0.003
log_return                        2.425e+06   1.91e+06      1.273      0.203   -1.31e+06    6.16e+06
%open                               -0.0009      0.003     -0.316      0.752      -0.006       0.004
open_xsinx2                          0.0486      0.064      0.757      0.449      -0.077       0.175
open_xsinxcosx2                      0.0509      0.067      0.756      0.450      -0.081       0.183
open_xtanx2                          0.0213      0.023      0.941      0.347      -0.023       0.066
high_phase                           0.0007      0.001      0.995      0.320      -0.001       0.002
high_sin                             0.0049      0.004      1.309      0.191      -0.002       0.012
high_sin2                           -0.1160      0.057     -2.044      0.041      -0.227      -0.005
high_xsinxcosx                       0.0081      0.010      0.776      0.438      -0.012       0.028
high_xsinx2                          0.2490      0.222      1.121      0.262      -0.187       0.685
high_xsinxcosx2                     -0.2946      0.417     -0.706      0.480      -1.113       0.524
high_xtanx2                          0.0045      0.020      0.221      0.825      -0.035       0.044
low_sin                              0.0005      0.002      0.251      0.802      -0.004       0.005
low_tan2                             0.1221      0.064      1.909      0.056      -0.003       0.248
low_xcosx                            0.0005      0.001      0.378      0.706      -0.002       0.003
low_xsinx2                          -0.1017      0.076     -1.331      0.183      -0.251       0.048
low_xsinxcosx2                      -0.1509      0.076     -1.977      0.048      -0.301      -0.001
low_xtanx2                          -0.0027      0.010     -0.278      0.781      -0.022       0.017
%close                           -2.384e+06   1.88e+06     -1.267      0.205   -6.07e+06     1.3e+06
mesa_close                          -1.8871      1.817     -1.039      0.299      -5.449       1.675
close_sin2                          -2.5247      2.433     -1.038      0.299      -7.294       2.245
close_tan                        -7.791e+04   8.34e+04     -0.935      0.350   -2.41e+05    8.55e+04
close_xcosx                         -0.0264      0.020     -1.354      0.176      -0.065       0.012
close_xsinxcosx                     -0.1770      0.062     -2.866      0.004      -0.298      -0.056
close_xcosx2                         0.4422      0.430      1.028      0.304      -0.401       1.286
close_xsinxcosx2                  -473.6283    456.60

In [31]:
lr = LogisticRegression(max_iter=10000)

lr.fit(X3_poly, ys)
lr.score(X3_poly, ys)

0.7638351172853564

In [32]:
pmask5 = results5.pvalues < 0.05
pmask5

1                  False
volume             False
log_return         False
%open              False
open_xsinx2        False
                   ...  
%low_29 %low_30     True
%low_29 %low_31    False
%low_30^2           True
%low_30 %low_31    False
%low_31^2          False
Length: 3570, dtype: bool

In [33]:
X4_poly = X3_poly[X3_poly.columns[pmask5]]

In [34]:
X4_poly.shape

(8782, 470)

In [35]:
model6 = sm.OLS(y, X4_poly)
results6 = model6.fit()
results6.summary()

<class 'statsmodels.iolib.summary.Summary'>
"""
                                 OLS Regression Results                                
=======================================================================================
Dep. Variable:                 target   R-squared (uncentered):                   0.282
Model:                            OLS   Adj. R-squared (uncentered):              0.242
Method:                 Least Squares   F-statistic:                              6.957
Date:                Mon, 07 Sep 2020   Prob (F-statistic):                        0.00
Time:                        20:01:00   Log-Likelihood:                          39911.
No. Observations:                8782   AIC:                                 -7.888e+04
Df Residuals:                    8312   BIC:                                 -7.556e+04
Df Model:                         470                                                  
Covariance Type:            nonrobust                                                  
================================================================================================
                                   coef    std err          t      P>|t|      [0.025      0.975]
------------------------------------------------------------------------------------------------
high_sin2                       -0.0019      0.000     -4.394      0.000      -0.003      -0.001
low_xsinxcosx2                  -0.0003      0.000     -1.149      0.250      -0.001       0.000
close_xsinxcosx                 -0.0083      0.005     -1.787      0.074      -0.017       0.001
%close_23                      6.39e-05   5.52e-05      1.158      0.247   -4.43e-05       0.000
%close_24                       -0.0004      0.000     -1.621      0.105      -0.001    8.68e-05
%high_16                      1.577e-05   3.38e-05      0.467      0.641   -5.05e-05     8.2e-05
%high_24                        -0.0002      0.000     -0.727      0.467      -0.001       0.000
%high_25                     -9.766e-06   3.46e-05     -0.282      0.778   -7.77e-05    5.81e-05
%high_26                        -0.0003      0.000     -1.595      0.111      -0.001    7.07e-05
%low_6                       -8.755e-05   6.63e-05     -1.321      0.187      -0.000    4.24e-05
volume^2                      1.523e-06   1.61e-06      0.944      0.345   -1.64e-06    4.69e-06
volume high_phase             -7.07e-05    4.1e-05     -1.725      0.085      -0.000    9.66e-06
volume low_sin               -5.216e-05   4.53e-05     -1.152      0.249      -0.000    3.66e-05
volume low_xsinx2            -2.674e-05      0.000     -0.057      0.955      -0.001       0.001
volume %open_sma30               0.0007      0.000      6.231      0.000       0.000       0.001
volume %open_sma50              -0.0006      0.000     -4.889      0.000      -0.001      -0.000
volume %close_22                 0.0002   4.79e-05      4.905      0.000       0.000       0.000
volume %close_28              2.912e-05   6.72e-05      0.434      0.665      -0.000       0.000
volume %close_29             -3.303e-05   5.02e-05     -0.658      0.511      -0.000    6.54e-05
volume %close_31                 0.0002   5.47e-05      3.904      0.000       0.000       0.000
volume %high_13                  0.0001   4.09e-05      2.462      0.014    2.05e-05       0.000
volume %high_14                 -0.0001    4.1e-05     -2.511      0.012      -0.000   -2.26e-05
volume %high_16               4.893e-05   3.92e-05      1.247      0.212    -2.8e-05       0.000
volume %high_28               4.907e-05   7.62e-05      0.644      0.520      -0.000       0.000
volume %high_30               3.496e-05   4.44e-05      0.787      0.431   -5.21e-05       0.000
volume %low_4                   -0.0001      5e-05     -2.383      0.017      -0.000   -2.12e-05
volume %low_6                   -0.0001   4.81e-05     -2.215      0.027      -0.000   -1.22e-05
volume %low_7                 2.149e-05   4.88e-05      0.440      0.660  

In [36]:
pmask6 = results6.pvalues < 0.05
X5 = X4_poly[X4_poly.columns[pmask6]]

model7 = sm.OLS(y, X5)
results7 = model7.fit()
results7.summary()

<class 'statsmodels.iolib.summary.Summary'>
"""
                                 OLS Regression Results                                
=======================================================================================
Dep. Variable:                 target   R-squared (uncentered):                   0.192
Model:                            OLS   Adj. R-squared (uncentered):              0.177
Method:                 Least Squares   F-statistic:                              13.04
Date:                Mon, 07 Sep 2020   Prob (F-statistic):                   3.29e-287
Time:                        20:01:13   Log-Likelihood:                          39390.
No. Observations:                8782   AIC:                                 -7.847e+04
Df Residuals:                    8625   BIC:                                 -7.735e+04
Df Model:                         157                                                  
Covariance Type:            nonrobust                                                  
================================================================================================
                                   coef    std err          t      P>|t|      [0.025      0.975]
------------------------------------------------------------------------------------------------
high_sin2                     6.583e-06      0.000      0.030      0.976      -0.000       0.000
volume %open_sma30               0.0007   9.63e-05      7.327      0.000       0.001       0.001
volume %open_sma50              -0.0006      0.000     -5.595      0.000      -0.001      -0.000
volume %close_22                 0.0002      4e-05      5.128      0.000       0.000       0.000
volume %close_31                 0.0002   4.65e-05      4.019      0.000    9.57e-05       0.000
volume %high_13              -2.258e-06   3.14e-05     -0.072      0.943   -6.38e-05    5.93e-05
volume %high_14                -9.4e-05   3.43e-05     -2.743      0.006      -0.000   -2.68e-05
volume %low_4                   -0.0001   4.51e-05     -2.905      0.004      -0.000   -4.26e-05
volume %low_6                -5.954e-05   4.24e-05     -1.404      0.160      -0.000    2.36e-05
log_return high_xtanx2           0.0185      0.004      4.648      0.000       0.011       0.026
log_return low_xcosx             0.0136      0.001     12.904      0.000       0.011       0.016
log_return close_xsinxcosx2     -0.0025      0.003     -0.762      0.446      -0.009       0.004
log_return %open_43              0.0034      0.005      0.660      0.509      -0.007       0.014
log_return %high_16              0.0132      0.003      3.932      0.000       0.007       0.020
%open close_kurt50           -7.808e-05   1.76e-05     -4.430      0.000      -0.000   -4.35e-05
%open %high_16                   0.0001   1.89e-05      6.801      0.000    9.13e-05       0.000
open_xsinxcosx2 %high_29     -1.461e-05   2.27e-05     -0.644      0.519   -5.91e-05    2.99e-05
open_xtanx2 %high_27            -0.0004   7.34e-05     -6.100      0.000      -0.001      -0.000
open_xtanx2 %low_7              -0.0005   7.28e-05     -7.255      0.000      -0.001      -0.000
open_xtanx2 %low_9              -0.0009      0.000     -4.697      0.000      -0.001      -0.001
open_xtanx2 %low_30             -0.0005      0.000     -3.172      0.002      -0.001      -0.000
open_xtanx2 %low_31              0.0007      0.000      3.987      0.000       0.000       0.001
high_phase high_xsinxcosx        0.0002   4.56e-05      3.301      0.001    6.12e-05       0.000
high_phase mesa_close        -3.286e-05   5.11e-05     -0.643      0.520      -0.000    6.73e-05
high_phase %close_1           5.899e-05   3.41e-05      1.732      0.083   -7.76e-06       0.000
high_phase %close_2          -8.682e-05   3.15e-05     -2.760      0.006      -0.000   -2.52e-05
high_phase %close_3           8.649e-05   4.94e-05      1.750      0.080   -1.04e-05       0.000
high_phase %high_19           9.299e-05   2.78e-05      3.351      0.001  

In [37]:
X5.shape

(8782, 157)

In [40]:
lr = LogisticRegression(max_iter=10000)

lr.fit(X4_poly, ys)
lr.score(X4_poly, ys)

0.6005465725347301

In [41]:
lr = LogisticRegression(max_iter=10000)

lr.fit(X5, ys)
lr.score(X5, ys)

0.5818720109314507

In [42]:
from sklearn.linear_model import LinearRegression

lr = LinearRegression()
lr.fit(X5, y)
lr.score(X5, y)

0.19161748353080066

In [126]:
X5

,log_return,high_xsinx2,high_xtanx2,low_sin,%close,mesa_close,close_xcosx2,close_sma30,volume mesa_open,volume open_xsinxcosx2,volume close_xcosx2,volume %open_4,volume %open_42,volume %close_21,volume close_sma30,volume %high_22,volume %low_22,volume low_sma30,log_return high_xsinx2,log_return low_xcosx,log_return low_xsinxcosx2,log_return low_xtanx2,log_return close_xsinxcosx,log_return open_kurt50,log_return %close_21,log_return close_kurt50,log_return %high_21,log_return low_sma30,mesa_open high_sin2,mesa_open high_xsinxcosx,mesa_open %low_3,open_xsinx2 high_angle,open_xsinx2 high_xsinxcosx,open_xsinx2 high_xsinx2,open_xsinx2 high_xtanx2,open_xsinx2 close_xsinxcosx,open_xsinx2 close_xcosx2,open_xsinx2 close_xsinxcosx2,open_xsinx2 %open_4,open_xsinx2 %open_43,open_xsinx2 %open_sma30,open_xsinx2 %close_2,open_xsinx2 %close_21,open_xsinx2 close_sma30,open_xsinx2 close_kurt50,open_xsinx2 %high_21,open_xsinx2 %low_sma30,open_xsinxcosx2 high_xsinxcosx,open_xsinxcosx2 high_xsinxcosx2,open_xsinxcosx2 close_xsinxcosx,open_xsinxcosx2 close_xsinxcosx2,open_xsinxcosx2 open_kurt50,open_xsinxcosx2 %close_3,open_xsinxcosx2 close_kurt50,open_xtanx2 close_xcosx2,open_xtanx2 %open_9,open_xtanx2 %close_1,open_xtanx2 %close_2,open_xtanx2 %high_21,high_phase high_xsinxcosx,high_phase high_xsinxcosx2,high_phase low_xsinxcosx2,high_phase close_xcosx,high_phase close_xcosx2,high_phase %open_43,high_phase open_kurt50,high_phase close_kurt50,high_angle high_xsinxcosx2,high_angle %low_23,high_sin2 low_xcosx,high_sin2 low_xsinxcosx2,high_sin2 %open_4,high_sin2 %open_9,high_sin2 %open_sma30,high_xsinxcosx high_xsinxcosx2,high_xsinxcosx %low,high_xsinxcosx low_sin,high_xsinxcosx low_xtanx2,high_xsinxcosx close_xsinxcosx,high_xsinxcosx %open_42,high_xsinxcosx %open_sma30,high_xsinx2 %low,high_xsinx2 mesa_low,high_xsinx2 low_sin,high_xsinx2 low_xtanx2,high_xsinx2 %close,high_xsinx2 mesa_close,high_xsinx2 close_xsinxcosx,high_xsinx2 close_xsinxcosx2,high_xsinx2 %open_41,high_xsinx2 %open_43,high_xcosx2 high_xtanx2,high_xcosx2 low_sin,high_xcosx2 close_xcosx,high_xcosx2 close_xcosx2,high_xcosx2 %open_4,high_xcosx2 %close_1,high_xcosx2 %close_2,high_xcosx2 close_kurt50,high_xcosx2 %high_21,...,%low^2,%low low_xcosx,%low close_xcosx2,%low open_kurt50,%low %open_sma30,%low %low_sma30,mesa_low low_sin,mesa_low close_xcosx2,mesa_low %open_4,mesa_low %close_2,low_sin low_xsinx2,low_sin low_xtanx2,low_sin %low_3,low_sin %low_sma30,low_tan2 low_xsinxcosx2,low_tan2 close_xcosx,low_tan2 close_xsinxcosx,low_tan2 close_xcosx2,low_xcosx^2,low_xcosx low_xsinxcosx2,low_xcosx %close,low_xcosx open_kurt50,low_xcosx %open_sma30,low_xcosx %low_sma30,low_xsinx2 low_xtanx2,low_xsinx2 close_xcosx,low_xsinx2 close_xcosx2,low_xsinx2 %open_4,low_xsinx2 close_sma30,low_xsinxcosx2 %close,low_xsinxcosx2 %open_41,low_xsinxcosx2 %close_1,low_xtanx2 %close,low_xtanx2 close_xsinxcosx,low_xtanx2 %open_4,low_xtanx2 open_kurt50,%close close_xsinxcosx,%close open_kurt50,%close %close_21,%close close_sma30,%close close_kurt50,%close %high_21,mesa_close %open_4,mesa_close %low_3,mesa_close low_sma30,close_sin2 %close_1,close_xcosx %open_9,close_xcosx %open_42,close_xcosx %open_43,close_xcosx open_kurt50,close_xcosx %close_1,close_xcosx close_sma30,close_xcosx %high_22,close_xcosx %low_28,close_xcosx low_sma30,close_xsinxcosx %open_9,close_xsinxcosx open_kurt50,close_xsinxcosx %low_22,close_xsinxcosx %low_23,close_xsinxcosx %low_sma30,close_xcosx2 %open_9,close_xcosx2 %open_42,close_xcosx2 %open_43,close_xcosx2 open_kurt50,close_xcosx2 %open_sma30,close_xcosx2 close_kurt50,close_xcosx2 %low_sma30,close_xsinxcosx2 %low_3,%open_4^2,%open_4 %open_9,%open_4 open_kurt50,%open_4 %close_1,%open_4 %high_23,%open_4 %low_sma30,%open_9^2,%open_9 %low_3,%open_41 %high_24,%open_42 %close_1,%open_43^2,open_kurt50 %close_21,open_kurt50 %high_21,open_kurt50 %high_23,open_kurt50 %high_24,open_kurt50 %low_23,%open_sma30 %close_3,%open_sma30 %close_21,%close_1 %high_23,%close_1 %high_24,%close_1 %low_3,%close_2 %

In [129]:
pmask8 = results7.pvalues < 0.05
X6 = X5[X5.columns[pmask8]]
X6

,log_return,high_xtanx2,low_sin,%close,close_xcosx2,close_sma30,volume mesa_open,volume open_xsinxcosx2,volume close_xcosx2,volume %open_42,volume %close_21,volume %high_22,volume %low_22,log_return low_xcosx,log_return low_xtanx2,log_return open_kurt50,log_return %close_21,log_return close_kurt50,log_return %high_21,log_return low_sma30,mesa_open high_sin2,open_xsinx2 high_angle,open_xsinx2 high_xsinxcosx,open_xsinx2 close_xcosx2,open_xsinx2 close_xsinxcosx2,open_xsinx2 %open_4,open_xsinx2 %close_21,open_xsinx2 close_sma30,open_xsinx2 %high_21,open_xsinxcosx2 open_kurt50,open_xsinxcosx2 close_kurt50,open_xtanx2 close_xcosx2,high_phase high_xsinxcosx,high_phase high_xsinxcosx2,high_phase low_xsinxcosx2,high_phase open_kurt50,high_phase close_kurt50,high_angle high_xsinxcosx2,high_angle %low_23,high_sin2 low_xcosx,high_sin2 low_xsinxcosx2,high_sin2 %open_9,high_sin2 %open_sma30,high_xsinxcosx high_xsinxcosx2,high_xsinxcosx low_xtanx2,high_xsinxcosx %open_sma30,high_xsinx2 %low,high_xsinx2 mesa_low,high_xsinx2 mesa_close,high_xsinx2 %open_41,high_xcosx2 low_sin,high_xcosx2 close_xcosx,high_xcosx2 close_xcosx2,high_xcosx2 %open_4,high_xcosx2 %close_1,high_xcosx2 %close_2,high_xcosx2 %high_21,high_xsinxcosx2 low_xtanx2,high_xsinxcosx2 close_sin2,high_xsinxcosx2 %open_43,high_xsinxcosx2 open_kurt50,high_xsinxcosx2 %close_1,high_xsinxcosx2 %low_3,high_xsinxcosx2 %low_sma30,high_xtanx2 %low,high_xtanx2 mesa_low,high_xtanx2 low_xcosx,high_xtanx2 low_xtanx2,high_xtanx2 mesa_close,high_xtanx2 %open_4,high_xtanx2 %close_2,high_xtanx2 close_kurt50,high_xtanx2 %high_22,high_xtanx2 %low_3,high_xtanx2 %low_22,high_xtanx2 %low_23,%low^2,%low low_xcosx,%low close_xcosx2,%low open_kurt50,%low %open_sma30,%low %low_sma30,mesa_low low_sin,mesa_low close_xcosx2,mesa_low %open_4,low_sin low_xtanx2,low_sin %low_3,low_xcosx^2,low_xcosx %close,low_xcosx open_kurt50,low_xcosx %open_sma30,low_xcosx %low_sma30,low_xsinx2 close_xcosx,low_xsinx2 close_xcosx2,low_xsinx2 %open_4,low_xsinxcosx2 %close_1,low_xtanx2 %close,low_xtanx2 close_xsinxcosx,low_xtanx2 %open_4,low_xtanx2 open_kurt50,%close open_kurt50,%close %close_21,%close close_sma30,%close close_kurt50,%close %high_21,mesa_close %open_4,mesa_close low_sma30,close_sin2 %close_1,close_xcosx %open_9,close_xcosx %open_42,close_xcosx %open_43,close_xcosx open_kurt50,close_xcosx %close_1,close_xcosx close_sma30,close_xcosx %high_22,close_xcosx %low_28,close_xcosx low_sma30,close_xsinxcosx %open_9,close_xsinxcosx %low_22,close_xsinxcosx %low_sma30,close_xcosx2 %open_9,close_xcosx2 %open_42,close_xcosx2 %open_43,close_xcosx2 open_kurt50,close_xcosx2 %open_sma30,close_xcosx2 %low_sma30,%open_4^2,%open_4 %open_9,%open_4 open_kurt50,%open_4 %close_1,%open_4 %high_23,%open_4 %low_sma30,%open_9^2,%open_9 %low_3,%open_41 %high_24,%open_42 %close_1,%open_43^2,open_kurt50 %close_21,open_kurt50 %high_21,open_kurt50 %high_23,open_kurt50 %high_24,open_kurt50 %low_23,%open_sma30 %close_21,%close_1 %high_24,%close_1 %low_3,%close_2 %high_21,close_sma30 close_kurt50,close_kurt50^2,close_kurt50 %high_21,close_kurt50 %high_24,close_kurt50 low_sma30,%high_21 %low_28,%high_22 %high_24
8844,-0.008060,0.004905,0.001151,-0.008028,0.007855,17.374333,97.986868,0.000143,1.571062,-0.118168,-0.229621,0.000000,-0.229358,0.000019,0.000048,0.011474,9.253636e-06,0.008020,1.385659e-05,-0.139907,-0.000702,0.017548,-6.593834e-12,5.637702e-09,-4.989036e-12,6.691779e-09,-8.239877e-10,0.000012,-1.233857e-09,-1.020333e-06,-7.131613e-07,-1.172115e-06,3.556144e-03,3.801288e-03,0.000911,551.016398,385.132724,-0.240121,0.000000,3.300078e-06,3.374866e-09,-0.000000e+00,5.147068e-06,9.022972e-11,5.517104e-08,3.298461e-08,-3.316650e-08,5.638545e-06,5.621956e-06,-6.803888e-09,7.885193e-06,4.346880e-05,5.381039e-05,6.387128e-05,3.553415e-05,1.186523e-05,-1.177684e-05,5.897428e-08,8.690907e-09,5.799163e-09,1.398089e-05,-5.094406e-08,-0.000000e+00,3.300345e-08,-1.413403e-05,0.002403,-1.128943e-05,-2.945152e-05,0.002396,4.572969e-05,8.495109e-06,-0.004880,

In [130]:
model8 = sm.OLS(y, X6)
results8 = model8.fit()
results8.summary()

<class 'statsmodels.iolib.summary.Summary'>
"""
                                 OLS Regression Results                                
=======================================================================================
Dep. Variable:                 target   R-squared (uncentered):                   0.247
Model:                            OLS   Adj. R-squared (uncentered):              0.234
Method:                 Least Squares   F-statistic:                              18.50
Date:                Mon, 07 Sep 2020   Prob (F-statistic):                        0.00
Time:                        19:39:08   Log-Likelihood:                          39700.
No. Observations:                8782   AIC:                                 -7.909e+04
Df Residuals:                    8629   BIC:                                 -7.801e+04
Df Model:                         153                                                  
Covariance Type:            nonrobust                                                  
==================================================================================================
                                     coef    std err          t      P>|t|      [0.025      0.975]
--------------------------------------------------------------------------------------------------
log_return                        71.2283      9.960      7.151      0.000      51.704      90.752
high_xtanx2                       -2.5979      0.292     -8.897      0.000      -3.170      -2.026
low_sin                           -9.3079      3.835     -2.427      0.015     -16.825      -1.791
%close                           -71.4231      9.967     -7.166      0.000     -90.961     -51.886
close_xcosx2                      19.9360      3.464      5.755      0.000      13.145      26.727
close_sma30                        0.0023      0.001      3.809      0.000       0.001       0.003
volume mesa_open                1.731e-08   4.03e-09      4.296      0.000    9.41e-09    2.52e-08
volume open_xsinxcosx2            -0.0003   4.93e-05     -5.321      0.000      -0.000      -0.000
volume close_xcosx2              3.18e-06   1.24e-06      2.563      0.010    7.48e-07    5.61e-06
volume %open_42                     1e-06   6.36e-07      1.574      0.116   -2.46e-07    2.25e-06
volume %close_21               -3.996e-06    7.9e-07     -5.057      0.000   -5.55e-06   -2.45e-06
volume %high_22                -3.447e-06   1.17e-06     -2.943      0.003   -5.74e-06   -1.15e-06
volume %low_22                  3.518e-06   1.18e-06      2.970      0.003     1.2e-06    5.84e-06
log_return low_xcosx            1835.0284    307.226      5.973      0.000    1232.791    2437.265
log_return low_xtanx2             64.7215     11.140      5.810      0.000      42.884      86.559
log_return open_kurt50             3.0836      0.372      8.298      0.000       2.355       3.812
log_return %close_21           -3431.4560    770.860     -4.451      0.000   -4942.525   -1920.387
log_return close_kurt50           -1.3556      0.223     -6.065      0.000      -1.794      -0.917
log_return %high_21             3722.7607    850.322      4.378      0.000    2055.926    5389.596
log_return low_sma30              -4.4763      0.673     -6.655      0.000      -5.795      -3.158
mesa_open high_sin2                0.0023      0.005      0.486      0.627      -0.007       0.012
open_xsinx2 high_angle            -0.0003   8.49e-05     -3.066      0.002      -0.000   -9.38e-05
open_xsinx2 high_xsinxcosx      3.506e+04   2.47e+04      1.419      0.156   -1.34e+04    8.35e+04
open_xsinx2 close_xcosx2        -575.4710    120.897     -4.760      0.000    -812.459    -338.483
open_xsinx2 close_xsinxcosx2    2179.8524    331.553      6.575      0.000    1529.929    2829.776
open_xsinx2 %open_4              -55.7726    130.424     -0.428      0.669    -311.435     199.890
open_xsinx2 %close_21           -135.6642    209.634     -0.647      0.518    -546.597     275.268
open_xsinx2 cl